In [1]:
import numpy as np
import pandas as pd

In [2]:
!ls ./contracts

ls: ./contracts: No such file or directory


In [3]:
data = pd.read_csv('./contracts/train/contract_60.csv')
data.head()

,xpaths,text,highlighted_xpaths,highlighted_segmented_text,tagged_sequence
0,/html/body/document/type,EX-2.1,NaN,NaN,o
1,/html/body/document/type/sequence,2,NaN,NaN,o
2,/html/body/document/type/sequence/filename,d19211dex21.htm,NaN,NaN,o
3,/html/body/document/type/sequence/filename/des...,EX-2.1,NaN,NaN,o
4,/html/body/document/type/sequence/filename/des...,EX-2.1,NaN,NaN,o


In [4]:
np.unique(data['tagged_sequence'])

array(['b_sst', 'b_st', 'b_t', 'e_sst', 'e_st', 'e_t', 'i_sst', 'i_t',
       'o', 's_n', 's_sn', 's_ssn', 's_sssn', 's_ssst', 's_sst', 's_st'],
      dtype=object)

In [8]:
data['highlighted_xpaths'].unique()

array([nan,
       '/html/body/document/type/sequence/filename/description/text/center[1]/div/p[5]',
       '/html/body/document/type/sequence/filename/description/text/center[1]/div/p[6]',
       '/html/body/document/type/sequence/filename/description/text/center[1]/div/p[7]',
       '/html/body/document/type/sequence/filename/description/text/center[1]/div/p[8]',
       '/html/body/document/type/sequence/filename/description/text/center[1]/div/p[9]',
       '/html/body/document/type/sequence/filename/description/text/center[1]/div/p[10]',
       '/html/body/document/type/sequence/filename/description/text/center[2]/div/p[1]',
       '/html/body/document/type/sequence/filename/description/text/center[2]/div/p[4]',
       '/html/body/document/type/sequence/filename/description/text/center[3]/div/p[1]',
       '/html/body/document/type/sequence/filename/description/text/center[3]/div/p[2]',
       '/html/body/document/type/sequence/filename/description/text/center[3]/div/p[6]',
       '

In [9]:
xpath = data['highlighted_xpaths'].unique()[1]

inds = np.asarray([False for i in range(data['xpaths'].shape[0])])

for i in range(data['xpaths'].shape[0]):
#     print(data['xpaths'][i])
    inds[i] = data['xpaths'][i].startswith(xpath)
    
a = data[inds]
a

,xpaths,text,highlighted_xpaths,highlighted_segmented_text,tagged_sequence
7,/html/body/document/type/sequence/filename/des...,AGREEMENT AND PLAN OF MERGER,/html/body/document/type/sequence/filename/des...,AGREEMENT AND PLAN OF MERGER,b_t


In [10]:
# data[(data['tagged_sequence'] == 's_ssn')]# + (data['tagged_sequence'] == 'e_sst')]

In [31]:
import json

json_file_path = 'maud_squad_train.json'

with open(json_file_path, 'r') as json_file:
    # Load and parse the JSON data
    data = json.load(json_file)


data = data['data'][0]
title = data['title']
qna = data['paragraphs'][0]
# print(data)
qas, context = qna['qas'], qna['context']


In [32]:
qas

[{'question': 'Highlight the parts of the text if any related to "Absence of Litigation Closing Condition" that should be reviewed by a lawyer.',
  'is_impossible': True,
  'answers': [],
  'id': 'contract_60_Absence of Litigation Closing Condition'},
 {'question': 'Highlight the parts of the text if any related to "Accuracy of Target R&W Closing Condition" that should be reviewed by a lawyer.',
  'is_impossible': False,
  'answers': [{'text': 'representation and warranty speaks as of a particular date, in which case such representation and warranty shall be true and correct, subject only to de minimis inaccuracies, as of such earlier date), (ii) the representation and warranty of the Company set forth in Section 5.1(f)(i) (Absence of Certain Changes) and Section 5.1(r) (Asset Management Agreement) shall be true and correct in all respects at the date hereof and the Closing, (iii) the representations and warranties of the Company set forth in the second and third sentence of Section 5.

In [23]:
answer_list = qas[1]['answers']
answer_list

[{'text': 'representation and warranty speaks as of a particular date, in which case such representation and warranty shall be true and correct, subject only to de minimis inaccuracies, as of such earlier date), (ii) the representation and warranty of the Company set forth in Section 5.1(f)(i) (Absence of Certain Changes) and Section 5.1(r) (Asset Management Agreement) shall be true and correct in all respects at the date hereof and the Closing, (iii) the representations and warranties of the Company set forth in the second and third sentence of Section 5.1(c) (Corporate Authority and Approval; Financial Advisor Opinion) and Section 5.1(s) (Brokers) shall be true and correct in all material respects, in each case, at the date hereof and the Closing (in each case except to the extent that such representation and warranty speaks as of a particular date, in which case such representation and warranty shall be true and correct in all material respects as of such earlier date)',
  'answer_s

In [25]:
len(answer_list)

9

## Highlighting

In [15]:
def highlight(file_name, output_file_name, start_string, end_string, color_tag = '#ffaf00'):
    with open(file_name,'r', encoding = 'latin1') as f:
        data = f.read()
#     print(len(data))
    for tag in ['<u>','</u>','<i>','</i>']:
        data = data.replace(tag, '')
        
#     print(data)
    
#     print(len(data))
    start = data.find(start_string)
    
    span_tag_start = f'<span style="display:inline; background-color:{color_tag}">'
    
    data = data[:start] + span_tag_start + data[start:]
#     ending = 'earlier date);'
    start = data.find(start_string)
    
    end_of_section = data[start:].find(end_string)
#     print(end_of_section)
    end = start + end_of_section + len(end_string)+1
    
    span_tag_end = '</span>'
    data = data[:end] + span_tag_end + data[end:]
    
    print(f'After adding the span tag, the html text is:\n\n\n{data[start:end+20]}')
    
    with open(output_file_name, 'w') as f:
        f.write(data)
    
#     print(len(data))
#     print(len(data))

file_name = 'sample1.html'
output_file_name = 'sample_highlighted.html'
start_string = 'representation and warranty'
end_string = 'earlier date);'
color_tag = '#ffaa00'
highlight(file_name, output_file_name, start_string, end_string, color_tag)


<!-- saved from url=(0082)https://www.sec.gov/Archives/edgar/data/1555039/000119312520274044/d19211dex21.htm -->
<html><head><meta http-equiv="Content-Type" content="text/html; charset=windows-1252"></head><body bgcolor="WHITE"><document>
<type>EX-2.1
<sequence>2
<filename>d19211dex21.htm
<description>EX-2.1
<text>

<title>EX-2.1</title>
<script>bazadebezolkohpepadr="1684781302"</script><script type="text/javascript" src="./sample1_files/646bb992" defer=""></script>
 


<center><div style="width:8.5in" align="left">
 <p style="margin-top:0pt; margin-bottom:0pt; font-size:10pt; font-family:Times New Roman" align="right"><b>Exhibit 2.1 </b></p>
<p style="margin-top:12pt; margin-bottom:0pt; font-size:10pt; font-family:Times New Roman" align="right"><b>Execution Version </b></p> <p style="font-size:12pt;margin-top:0pt;margin-bottom:0pt">&nbsp;</p>
<p style="line-height:1.0pt;margin-top:0pt;margin-bottom:2pt;border-bottom:1px solid #000000">&nbsp;</p> <p style="margin-top:12pt; margin-bott

In [33]:
data[data['tagged_sequence'] == 'e_st']

,xpaths,text,highlighted_xpaths,highlighted_segmented_text,tagged_sequence
351,/html/body/document/type/sequence/filename/des...,ERGER,/html/body/document/type/sequence/filename/des...,ERGER,e_st
364,/html/body/document/type/sequence/filename/des...,ECITALS,/html/body/document/type/sequence/filename/des...,ECITALS,e_st


In [29]:
to_highlight = data[data['tagged_sequence'] == 'e_st']['xpaths'].values

In [24]:
data[data['text'] == 'ECITALS']

,xpaths,text,highlighted_xpaths,highlighted_segmented_text,tagged_sequence
364,/html/body/document/type/sequence/filename/des...,ECITALS,/html/body/document/type/sequence/filename/des...,ECITALS,e_st


In [9]:
contract_to_html = pd.read_csv('./contracts/Contract Name to HTML Link Mapping.csv')
contract_to_html

,Name,Link to EDGAR Filing,Name in Dataset
0,Acacia_Communications_Cisco_Systems,https://www.sec.gov/Archives/edgar/data/165123...,contract_0
1,Acceleron_Pharma_Inc_Merck_Co,https://www.sec.gov/Archives/edgar/data/128060...,contract_1
2,Adamas_Pharmaceuticals_Supernus_Pharmaceuticals,https://www.sec.gov/Archives/edgar/data/132814...,contract_2
3,"Aegion Corporation_New Mountain Capital, L.L.C.",https://www.sec.gov/Archives/edgar/data/353020...,contract_3
4,"Alaska Communications Systems Group, Inc._Inve...",https://www.sec.gov/Archives/edgar/data/108951...,contract_4
...,...,...,...
147,"Waddell _ Reed Financial, Inc._Macquarie Group...",https://www.sec.gov/Archives/edgar/data/105210...,contract_147
148,Weingarten Realty Investors_Kimco Realty Corpo...,https://www.sec.gov/Archives/edgar/data/828916...,contract_148
149,Weingarten Realty Investors_Kimco Realty Corpo...,https://www.sec.gov/Archives/edgar/data/828916...,contract_149
150,Advanced Micro Devices,https://www.sec.gov/Archives/edgar/data/2488/0...,contract_150


In [10]:
contract_to_html['Link to EDGAR Filing'].iloc[0]

'https://www.sec.gov/Archives/edgar/data/1651235/000119312521008768/d110570dex21.htm'

In [13]:
import requests

# URL of the PDF you want to download
pdf_url = 'https://www.sec.gov/Archives/edgar/data/1651235/000119312521008768/d110570dex21.htm'

# Send an HTTP GET request to the URL
response = requests.get(pdf_url)
print(response)

if response.status_code == 200:
    # The request was successful, and the content is in response.content
    with open("./downloaded.pdf", "wb") as pdf_file:
        pdf_file.write(response.content)
    print("PDF downloaded successfully.")
else:
    print(f"Failed to download PDF. Status code: {response.status_code}")


<Response [403]>
Failed to download PDF. Status code: 403


In [14]:
!pip install PyMuPDF

  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/7e/93/c00b4259925c6a2cc752a28e349bc89795354f7dbb235255ca93a2ba7f2a/PyMuPDF-1.23.4-cp310-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for PyMuPDFb==1.23.3 from https://files.pythonhosted.org/packages/2a/42/b240c953c0ca57355a83978a90a5681e68c6da7d8e8212e25c8b029eddf8/PyMuPDFb-1.23.3-py3-none-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 38.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 52.6 MB/s eta 0:00:0000:0100:01


In [40]:
import fitz  # PyMuPDF
import sys

# Define a function to highlight a specific line in the PDF
def highlight_line(pdf_path, output_path, line_number, page_number, highlight_color=(1, 0, 0)):
    doc = fitz.open(pdf_path)
    page = doc[page_number]  # Assuming you want to highlight a line on the first page
    lines = page.get_text("words")
#     print(lines)
    if 0 < line_number <= len(lines):
        selected_line = lines[line_number - 1]
        x0, y0, x1, y1 = selected_line[:4]
        highlight = page.add_highlight_annot((x0, y0, x1, y1))
        highlight.update(fill_color=highlight_color)

        doc.save(output_path)
        doc.close()
    else:
        print("Invalid line number. Please provide a valid line number within the range.")


In [44]:
pdf_path = './sample.pdf'
output_path = './sample_coloured.pdf'
line_number = 64
page_number = 5

highlight_line(pdf_path, output_path, line_number, page_number)

In [31]:
!pip install beautifulsoup4

In [34]:
import requests
from bs4 import BeautifulSoup

# URL of the web page with XML content
url = "file:///Users/anshriyal/Downloads/Capstone/MarkupMnA-Markup-Based-Segmentation-of-MnA-Agreements/sample.html"

# Send an HTTP GET request to the URL
response = requests.get(url)

if response.status_code == 200:
    # Parse the content using BeautifulSoup with XML parser
    soup = BeautifulSoup(response.text, 'xml')

    # Print the XML content or do further processing
    xml_content = soup.prettify()
    print(xml_content)
else:
    print(f"Failed to fetch XML. Status code: {response.status_code}")


InvalidSchema: No connection adapters were found for 'file:///Users/anshriyal/Downloads/Capstone/MarkupMnA-Markup-Based-Segmentation-of-MnA-Agreements/sample.html'

In [35]:
maud = pd.read_csv('./../maud/data/raw/main.csv')
maud

,Filename,Filename (anon),Type of Consideration,Type of Consideration-Answer,Stock Deal: Fixed Ratio v. Fixed Value-Answer,Accuracy of Target R&W Closing Condition,"Accuracy of Target ""General"" R&W: Bringdown Timing Answer","Accuracy of Target ""General"" R&W: Bringdown Standard Answer",Accuracy of Target Capitalization R&W (outstanding shares): Bringdown Standard Answer,Accuracy of Fundamental Target R&Ws: Bringdown Standard,...,Negative interim operating covenant,Buyer consent requirement (negative interim covenant)-Answer,Application of Buyer consent requirement (negative interim covenant)-Answer,Negative Interim Covenant includes carveout for pandemic responses-Answer (Y/N),General Antitrust Efforts Standard,General Antitrust Efforts Standard-Answer,Limitations on Antitrust Efforts,Limitations on Antitrust Efforts-Answer,Specific Performance,Specific Performance-Answer
0,"The Michaels Companies, Inc._Apollo Global Man...",contract_35.pdf,"WHEREAS, pursuant to this Agreement, Merger Su...",All Cash,(None entered),Conditions of the Offer Notwithstanding any ot...,At Signing & At Closing,All/The R&Ws accurate at MAE standard,Accurate in all respects with de minimis excep...,Accurate in all material respects,...,Except for matters set forth in Section 6.01 o...,"Consent may not be unreasonably withheld, cond...",Applies to all negative covenants,No,Section 8.01. Efforts. \n\n\n(a) ...,Flat standard,(None entered),"""Hell or high water"" standard (no limitation &...",Section 11.14. Specific Performance. The parti...,"""entitled to"" specific performance"
1,TIFFANY_&_CO._LVMH_MOËT_HENNESSY-LOUIS_VUITTON...,contract_37.pdf,"WHEREAS, the Parties intend to effect the acqu...",All Cash,(None entered),ARTICLE V REPRESENTATIONS AND WARRANTIES OF TH...,(None entered),(None entered),Accurate in all respects with de minimis excep...,Accurate in all respects,...,7.1 Conduct of the Company. <omitted> Without ...,"Consent may not be unreasonably withheld, cond...",Applies to all negative covenants,Yes,7.4 Proxy Statement; Other Regulatory Matters....,Flat standard,(None entered),"""Hell or high water"" standard (no limitation &...",10.6 Specific Performance; Remedies. \n\n(a) E...,"""entitled to seek"" specific performance"
2,Inovalon_Holdings_Management_Led_Buyout.pdf,contract_64.pdf,Section 3.01 Effect of Merger on Capital Stock...,All Cash,(None entered),Section 4.02 Corporate Authorization. <omitted...,At Signing & At Closing,All/The R&Ws accurate at MAE standard,Accurate in all respects with de minimis excep...,Accurate at another materiality standard (e.g....,...,"(a) The Company agrees that, from the date of ...","Consent may not be unreasonably withheld, cond...",Applies to all negative covenants,No,Section 6.03 Appropriate Action; Consents; Fil...,Flat standard,(None entered),"""Hell or high water"" standard (no limitation &...",Section 9.02 Remedies Cumulative; Specific Per...,"""entitled to"" specific performance"
3,Cubic Corporation_Investment Group.pdf\n\nCubi...,contract_79.pdf,(i) Conversion of Company Common St...,All Cash,(None entered),Section 6.02 Additional Conditions to Obligati...,At Signing & At Closing,All/The R&Ws accurate at MAE standard,Accurate in all respects with de minimis excep...,Accurate in all material respects,...,"Without limiting the foregoing, and as an exte...","Consent may not be unreasonably withheld, cond...",Applies to all negative covenants,Yes,Section 5.06 Appropriate Action; Con...,Flat standard,(None entered),"""Hell or high water"" standard (no limitation &...",Section 8.10 Specific Performance. (a) ...,"""entitled to"" specific performance"
4,HMS Holdings Corp._Veritas Capital.pdf,contract_81.pdf,(a) Conversion of Company Common...,All Cash,(None entered),Section 8.02 Conditions to the Obligatio...,At Signing & At Closing,Each R&W accurate at MAE standard,Accurate in all material respects,Accurate in all material respects,...,"provided, however, that the failure by an Acqu...","Consent may